In [1]:
import cv2 as cv
import os
from os.path import exists
import numpy as np
import mediapipe as mp

objc[25768]: Class CaptureDelegate is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x16b7765a0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x137a1c860). One of the two will be used. Which one is undefined.
objc[25768]: Class CVWindow is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x16b7765f0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x127384a68). One of the two will be used. Which one is undefined.
objc[25768]: Class CVView is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x16b776618) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x127384a90). One of the two will be used. Which one is undefined.
objc[25768]: Class CVSlider is implemented in both /Users/M

In [2]:
def extractKeypoints(result):
    if result.multi_face_landmarks:
        for face_detected in result.multi_face_landmarks:
            face = np.array([[res.x, res.y, res.z] for res in face_detected.landmark]).flatten()
    else:
        face = np.zeros(1434)

    return face

def drawLandmarks(mp_face_mesh, results, frame): 
    mp_drawing = mp.solutions.drawing_utils  
    mp_drawing_styles = mp.solutions.drawing_styles
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
                                       connections=mp_face_mesh.FACEMESH_TESSELATION, 
                                       landmark_drawing_spec=None,
                                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
                                       connections=mp_face_mesh.FACEMESH_CONTOURS,
                                         landmark_drawing_spec=None,
                                           connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks,
                                       connections=mp_face_mesh.FACEMESH_IRISES,
                                         landmark_drawing_spec=None,
                                           connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())
            
def revertColors(frame, mesh): 
    image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  
    image.flags.writeable = False 
    results = mesh.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)  
    
    return image, results 

## Extract Nodding values

In [3]:
DATA_PATH = os.path.join('./Nodding_Extracted_Values//')

for state in ["Nodding", "NonNodding"]:
    for number in range(1,91):
            videoIndex  = str(number).zfill(3)
            try:
                os.makedirs(os.path.join(DATA_PATH, state , str(videoIndex)))
            except:
                pass

In [4]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

videoNumberRange = [1, 36, 2, 6, 8, 9, 12, 13, 15, 20, 23, 24, 31, 32, 33, 34, 35, 5]

saveVideoIndex = 1
for videoNumber in videoNumberRange:
    for state in ["glasses", "noglasses", "night_noglasses", "sunglasses", "nightglasses"]:
        videoIndex = str(videoNumber).zfill(3)
        videoPath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                 videoIndex + "/" + state + "/slowBlinkWithNodding.avi")
        cap = cv.VideoCapture(videoPath)
        textFileName = "_slowBlinkWithNodding_head.txt"
        textFilePath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                    videoIndex + "/" + state + "/" + videoIndex + textFileName)
        print("Processing video number: ", str(videoNumber), " for state ", state, " Save Index:  ", saveVideoIndex)

        with open(textFilePath, 'r') as file:
            noddingFrameCounter = 0
            nonNoddingFrameCounter = 0
            for char in file.read():
                if char == "1":
                    savePath = os.path.join("./Nodding_Extracted_Values/Nodding/", str(saveVideoIndex).zfill(3), str(noddingFrameCounter))
                    noddingFrameCounter += 1
                else:
                    savePath = os.path.join("./Nodding_Extracted_Values/NonNodding/", str(saveVideoIndex).zfill(3), str(nonNoddingFrameCounter))
                    nonNoddingFrameCounter += 1

                with mp_face_mesh.FaceMesh(
                                refine_landmarks=True,
                                min_detection_confidence=0.6,
                                min_tracking_confidence=0.7) as mesh:
                    
                    ret, frame = cap.read()               
                    image, results = revertColors(frame, mesh)
                    drawLandmarks(mp_face_mesh, results, image)
                    landmarks = extractKeypoints(results)

                    np.save(savePath, landmarks)
                    cv.imshow('Video', image)
                                    
                    if cv.waitKey(1) == ord('q'):
                        break              

        cap.release()
        saveVideoIndex += 1
cv.destroyAllWindows()

Processing video number:  1  for state  glasses  Save Index:   1


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Processing video number:  1  for state  noglasses  Save Index:   2
Processing video number:  1  for state  night_noglasses  Save Index:   3
Processing video number:  1  for state  sunglasses  Save Index:   4
Processing video number:  1  for state  nightglasses  Save Index:   5
Processing video number:  36  for state  glasses  Save Index:   6
Processing video number:  36  for state  noglasses  Save Index:   7
Processing video number:  36  for state  night_noglasses  Save Index:   8
Processing video number:  36  for state  sunglasses  Save Index:   9
Processing video number:  36  for state  nightglasses  Save Index:   10
Processing video number:  2  for state  glasses  Save Index:   11
Processing video number:  2  for state  noglasses  Save Index:   12


[NULL @ 0x2ccad8ae0] time_scale/num_units_in_tick invalid or unsupported (0/0)
[NULL @ 0x2ccad8ae0] Overread VUI by 8 bits
[h264 @ 0x2cd623610] Invalid level prefix
[h264 @ 0x2cd623610] error while decoding MB 5 4
[h264 @ 0x2cd623610] cbp too large (3199971767) at 36 25
[h264 @ 0x2cd623610] error while decoding MB 36 25
[h264 @ 0x2cd61ed60] illegal short term buffer state detected


Processing video number:  2  for state  night_noglasses  Save Index:   13
Processing video number:  2  for state  sunglasses  Save Index:   14
Processing video number:  2  for state  nightglasses  Save Index:   15
Processing video number:  6  for state  glasses  Save Index:   16
Processing video number:  6  for state  noglasses  Save Index:   17
Processing video number:  6  for state  night_noglasses  Save Index:   18
Processing video number:  6  for state  sunglasses  Save Index:   19
Processing video number:  6  for state  nightglasses  Save Index:   20
Processing video number:  8  for state  glasses  Save Index:   21
Processing video number:  8  for state  noglasses  Save Index:   22


...

[NULL @ 0x2cf4a6130] time_scale/num_units_in_tick invalid or unsupported (0/0)
[NULL @ 0x2cf4a6130] Overread VUI by 8 bits
[h264 @ 0x2cf4c96d0] Invalid level prefix
[h264 @ 0x2cf4c96d0] error while decoding MB 0 10
[h264 @ 0x2cf4c96d0] Invalid level prefix
[h264 @ 0x2cf4c96d0] error while decoding MB 15 21
[h264 @ 0x2cf4b9a40] illegal short term buffer state detected


Processing video number:  35  for state  nightglasses  Save Index:   85
Processing video number:  5  for state  glasses  Save Index:   86
Processing video number:  5  for state  noglasses  Save Index:   87
Processing video number:  5  for state  night_noglasses  Save Index:   88
Processing video number:  5  for state  sunglasses  Save Index:   89
Processing video number:  5  for state  nightglasses  Save Index:   90


OpenCV: Couldn't read video stream from file "/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/005/nightglasses/slowBlinkWithNodding.avi"
[ERROR:0@7095.287] global cap.cpp:166 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): /Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/005/nightglasses/slowBlinkWithNodding.avi in function 'icvExtractPattern'




FileNotFoundError: [Errno 2] No such file or directory: '/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/005/nightglasses/005_slowBlinkWithNodding_head.txt'

: 

## Extract Yawning values

In [ ]:
DATA_PATH = os.path.join('./Yawning//')

for state in ["Yawning", "NonYawning"]:
    for number in range(1,91):
            videoIndex  = str(number).zfill(3)
            try:
                os.makedirs(os.path.join(DATA_PATH, state , str(videoIndex)))
            except:
                pass

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

videoNumberRange = [1, 36, 2, 6, 8, 9, 12, 13, 15, 20, 23, 24, 31, 32, 33, 34, 35, 5]

saveVideoIndex = 1
for videoNumber in videoNumberRange:
    for state in ["glasses", "noglasses", "night_noglasses", "sunglasses", "nightglasses"]:
        videoIndex = str(videoNumber).zfill(3)
        videoPath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                 videoIndex + "/" + state + "/yawning.avi")
        cap = cv.VideoCapture(videoPath)
        textFileName = "_yawning_mouth.txt"
        textFilePath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                    videoIndex + "/" + state + "/" + videoIndex + textFileName)
        print("Processing video number: ", str(videoNumber), " for state ", state, " Save Index:  ", saveVideoIndex)

        with open(textFilePath, 'r') as file:
            yawningFrameCounter = 0
            nonYawningFrameCounter = 0
            for char in file.read():
                if char == "1":
                    savePath = os.path.join("./Yawning/Yawning/", str(saveVideoIndex).zfill(3), str(yawningFrameCounter))
                    yawningFrameCounter += 1
                else:
                    savePath = os.path.join("./Yawning/NonYawning/", str(saveVideoIndex).zfill(3), str(nonYawningFrameCounter))
                    nonYawningFrameCounter += 1

                with mp_face_mesh.FaceMesh(
                                refine_landmarks=True,
                                min_detection_confidence=0.6,
                                min_tracking_confidence=0.7) as mesh:
                    
                    ret, frame = cap.read()               
                    image, results = revertColors(frame, mesh)
                    drawLandmarks(mp_face_mesh, results, image)
                    landmarks = extractKeypoints(results)

                    np.save(savePath, landmarks)
                    cv.imshow('Video', image)
                                    
                    if cv.waitKey(1) == ord('q'):
                        break              

        cap.release()
        saveVideoIndex += 1
cv.destroyAllWindows()

## Extract Eyes values

In [ ]:
DATA_PATH = os.path.join('./Eyes_Extracted_Values//')

for state in ["Sleepy", "NonSleepy"]:
    for number in range(1,91):
            videoIndex  = str(number).zfill(3)
            try:
                os.makedirs(os.path.join(DATA_PATH, state , str(videoIndex)))
            except:
                pass

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

videoNumberRange = [1, 36, 2, 6, 8, 9, 12, 13, 15, 20, 23, 24, 31, 32, 33, 34, 35, 5]

saveVideoIndex = 1
for videoNumber in videoNumberRange:
    for state in ["glasses", "noglasses", "night_noglasses", "sunglasses", "nightglasses"]:
        videoIndex = str(videoNumber).zfill(3)
        videoPath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                 videoIndex + "/" + state + "/slowBlinkWithNodding.avi")
        cap = cv.VideoCapture(videoPath)
        textFileName = "_slowBlinkWithNodding_eye.txt"
        textFilePath = os.path.join('/Users/M374155/Desktop/New Dataset/Training_Evaluation_Dataset/Training Dataset/' +
                                    videoIndex + "/" + state + "/" + videoIndex + textFileName)
        print("Processing video number: ", str(videoNumber), " for state ", state, " Save Index:  ", saveVideoIndex)

        with open(textFilePath, 'r') as file:
            sleepyEyesFrameCounter = 0
            nonSleepyEyesFrameCounter = 0
            for char in file.read():
                if char == "1":
                    savePath = os.path.join("./Eyes_Extracted_Values/Sleepy/", str(saveVideoIndex).zfill(3), str(sleepyEyesFrameCounter))
                    sleepyEyesFrameCounter += 1
                else:
                    savePath = os.path.join("./Eyes_Extracted_Values/NonSleepy/", str(saveVideoIndex).zfill(3), str(nonSleepyEyesFrameCounter))
                    nonSleepyEyesFrameCounter += 1

                with mp_face_mesh.FaceMesh(
                                refine_landmarks=True,
                                min_detection_confidence=0.6,
                                min_tracking_confidence=0.7) as mesh:
                    
                    ret, frame = cap.read()               
                    image, results = revertColors(frame, mesh)
                    drawLandmarks(mp_face_mesh, results, image)
                    landmarks = extractKeypoints(results)

                    np.save(savePath, landmarks)
                    cv.imshow('Video', image)
                                    
                    if cv.waitKey(1) == ord('q'):
                        break              

        cap.release()
        saveVideoIndex += 1
cv.destroyAllWindows()